In [18]:
import requests
import re
import pandas as pd
import os
from datetime import date
import datetime
import json

# Settings

In [19]:
read_weather_from_file = False

# Read the data

In [20]:
df_sc = pd.read_csv(os.path.join("data", "sc_export.csv"))

# Parse timestamps

In [21]:
#date,hour = zip(*df_sc['timestamp'].map(lambda x: x.split('T')))
df_sc['timestamp'] = df_sc['timestamp'].map(lambda d:datetime.datetime.strptime(d,'%Y-%m-%dT%H:%M:%S'))

In [22]:
df_sc['day'] = df_sc['timestamp'].map(date.weekday)
df_sc['hour'] = pd.DatetimeIndex(df_sc['timestamp']).hour
df_sc['date'] = pd.DatetimeIndex(df_sc['timestamp']).date
df_sc['time'] = pd.DatetimeIndex(df_sc['timestamp']).hour + pd.DatetimeIndex(df_sc['timestamp']).minute/60 + pd.DatetimeIndex(df_sc['timestamp']).second/3600
df_sc['weekend'] = [1 if d == 5 or d==6 else 0 for d in df_sc['day']]

In [23]:
df_sc

,sensor_id,sensor_type,location,lat,lon,timestamp,P1,durP1,ratioP1,P2,durP2,ratioP2,day,hour,date,time,weekend
0,74650,SDS011,63169,46.042,14.592,2022-09-01 00:04:32,4.45,NaN,NaN,2.28,NaN,NaN,3,0,2022-09-01,0.075556,0
1,74650,SDS011,63169,46.042,14.592,2022-09-01 00:07:00,5.18,NaN,NaN,3.33,NaN,NaN,3,0,2022-09-01,0.116667,0
2,74650,SDS011,63169,46.042,14.592,2022-09-01 00:09:25,5.85,NaN,NaN,3.13,NaN,NaN,3,0,2022-09-01,0.156944,0
3,74650,SDS011,63169,46.042,14.592,2022-09-01 00:11:52,5.95,NaN,NaN,4.10,NaN,NaN,3,0,2022-09-01,0.197778,0
4,74650,SDS011,63169,46.042,14.592,2022-09-01 00:14:19,5.28,NaN,NaN,2.63,NaN,NaN,3,0,2022-09-01,0.238611,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49141,67867,SDS011,54672,46.096,14.470,2022-10-25 23:49:16,29.60,NaN,NaN,7.72,NaN,NaN,1,23,2022-10-25,23.821111,0
49142,67867,SDS011,54672,46.096,14.470,2022-10-25 23:51:41,35.90,NaN,NaN,7.47,NaN,NaN,1,23,2022-10-25,23.861389,0
49143,67867,SDS011,54672,46.096,14.470,2022-10-25 23:54:07,71.68,NaN,NaN,9.40,NaN,NaN,1,23,2022-10-25,23.901944,0
49144,67867,SDS011,54672,46.096,14.470,2022-10-25 23:56:33,20.30,NaN,NaN,7.18,NaN,NaN,1,23,2022-10-25,23.942500,0


In [24]:
#df_sc['name'] = [str(lat)+","+str(lon) for lat, lon in zip(df_sc['lat'], df_sc['lon'])]

# Select relevant columns

In [25]:
selected = ["sensor_id", 
            "sensor_type", 
            "location", 
            "lat", "lon", 
            "timestamp", 
            "date", 
            "time", "hour", 
            "day", "weekend", 
            "P1", "P2"]

df = df_sc[selected]

# Get weather data

In [26]:
weather_groups = {'Clear': 'dry',
                  'Partially cloudy': 'dry',
                  'Rain, Partially cloudy': 'humid',
                  'Rain': 'wet',
                  'Rain, Overcast': 'wet',
                  'Overcast': 'humid'}

In [34]:
if not read_weather_from_file:
    with open('api_keys.json') as f:
        API_KEY = json.loads(f.read())['weather']

    lat_lon_s_id = set(zip(df_sc.lat, df_sc.lon, df_sc.sensor_id))
    days = df_sc.date.unique()

    df_weather = -1

    for lat,lon,sensor_id in lat_lon_s_id:
        min_day = min(days)
        max_day = max(days)
        print(min_day, max_day, lat, lon)
        try:
            url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat}%2C{lon}/{min_day}/{max_day}?unitGroup=metric&include=days&key={API_KEY}&contentType=csv"

            df2 = pd.read_csv(url)
            df2['sensor_id'] = sensor_id
            if type(df_weather) == int:
                df_weather = df2
            else:
                df_weather = pd.concat([df_weather, df2],ignore_index=True)
        except:
            print("Something is wrong...")

    df_weather.to_csv(os.path.join("data", "weather.csv"), index=False)
else:
    df_weather = pd.read_csv(os.path.join("data", "weather.csv"))


2022-09-01 2022-10-25 46.042 14.592
2022-09-01 2022-10-25 46.096 14.47


In [35]:
df_weather['weather'] = [weather_groups[x] for x in df_weather["conditions"]]

In [36]:
df_weather.rename(columns={"datetime": "date"}, inplace=True)

In [37]:
df_weather['date'] = pd.DatetimeIndex(df_weather['date']).date

In [38]:
selected = ["sensor_id","date", "temp", "tempmax", "tempmin", "conditions", "weather"]

In [39]:
df_weather = df_weather[selected]

In [40]:
df_weather

,sensor_id,date,temp,tempmax,tempmin,conditions,weather
0,74650,2022-09-01,15.8,17.6,13.7,"Rain, Partially cloudy",humid
1,74650,2022-09-02,16.1,22.3,8.4,Partially cloudy,dry
2,74650,2022-09-03,17.7,24.6,9.7,Partially cloudy,dry
3,74650,2022-09-04,18.9,27.6,11.5,"Rain, Partially cloudy",humid
4,74650,2022-09-05,19.2,27.6,11.1,"Rain, Partially cloudy",humid
...,...,...,...,...,...,...,...
105,67867,2022-10-21,13.4,18.1,9.2,"Rain, Partially cloudy",humid
106,67867,2022-10-22,14.5,16.4,11.7,"Rain, Partially cloudy",humid
107,67867,2022-10-23,13.0,16.7,10.2,"Rain, Partially cloudy",humid
108,67867,2022-10-24,15.5,19.8,11.8,Partially cloudy,dry


# Add weather to sensor data

In [41]:
df = pd.merge(df, df_weather, how='left', on=['sensor_id', 'date'])

In [42]:
df.to_csv(os.path.join("data", "sc_weather.csv"), index=False)